# Chapter 2: Automating the betting system

In this chapter we will create a function that automatically calculates the team's odds of winning based on the American odds number introduced in the intro []`content:gambling`. 

In [8]:
![gambling.jpg]

/bin/bash: [gambling.jpg]: command not found


We want to create a function that calculates how much you win based on the American odds and the amount wagered. 

In [24]:
def win_calc(a,b):
#a is the odds and b is the amount bet
    if a > 0:
        win = (b/(abs(a)))*100
    else:
        win = (b/100)* (abs(a))
        print(win)

In [25]:
odds_calc(-100,20)

20.0
